* Pix2Pixを使って，ピザの輪郭線からピザの画像を生成

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# カレントディレクトリの読み込みとカレントディレクトリへの移動
import sys
sys.path.append(f'/content/drive/My Drive/system/')
import os
os.chdir(f'/content/drive/My Drive/system/myanswer')

In [3]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-h1gvrft1
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-h1gvrft1
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=f439da0b8cccd65bc94fdaf54e303570ea93d7b1c0238e4caa1c45f1b5174e09
  Stored in directory: /tmp/pip-ephem-wheel-cache-7uqlbx3q/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install scipy==1.1.0

In [5]:
from __future__ import print_function, division

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from glob import glob
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.misc

In [6]:
class DataGenerator():
      def __init__(self, target_name, dataset_name, img_res=(128, 128)):
        self.target_name = target_name # pix2pixで変換を行う食事名
        self.dataset_name = dataset_name 
        self.img_res = img_res

      def generate_data(self, indices=1000, train_percentage=0.6, val_percentage=0.2):
         """
          食事画像と輪郭線画像を横に連結した画像を生成する関数
          indices(int): 画像枚数

          デフォルトでは，self.target_nameで指定した画像のうち
          6割 (train_percentage): 学習データ用の画像
          2割 (val_percentage): 検証データ用の画像
          2割: テストデータ用の画像
          とする
         """
         os.makedirs('../figure/foodimg128/{}/train'.format(self.dataset_name), exist_ok=True)
         os.makedirs('../figure/foodimg128/{}/val'.format(self.dataset_name), exist_ok=True)
         os.makedirs('../figure/foodimg128/{}/test'.format(self.dataset_name), exist_ok=True)

         # self.target_nameで指定された食事画像のパスを取得
         image_pathes = glob('../figure/foodimg128/{}/*.jpg'.format(self.target_name))
         # self.target_nameで指定された輪郭線画像のパスを取得
         contour_pathes = glob('../figure/foodimg128/{}/*.png'.format(self.target_name))

         for index in range(indices):
            image = Image.open(image_pathes[index])
            contour = Image.open(contour_pathes[index])
            new_image = Image.new('RGB', (image.width + contour.width, min(image.height, contour.height)))
            new_image.paste(image, (0, 0))
            new_image.paste(contour, (image.width, (image.height - contour.height) // 2)) # 連結した画像を中央に配置
            if indices * train_percentage <= index < indices * (train_percentage + val_percentage):
                new_image.save("../figure/foodimg128/{}/val/{}.jpg".format(self.dataset_name, index)) # 検証データの保存
            elif indices * (train_percentage + val_percentage) <= index < indices:
                new_image.save("../figure/foodimg128/{}/test/{}.jpg".format(self.dataset_name, index)) # テストデータの保存
            else:
                new_image.save("../figure/foodimg128/{}/train/{}.jpg".format(self.dataset_name, index)) # 学習データの保存               

In [7]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res
    
    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('../figure/foodimg128/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('../figure/foodimg128/%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [8]:
class Pix2Pix():
    def __init__(self):
        # Number of images　(簡単のため，入力に使う画像枚数を既知であることが前提)
        self.image_num = 1000
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data generator and loader
        self.dataset_name = 'pix2pix_pizza'
        self.target_name = "pizza"
        self.data_generator = DataGenerator(target_name=self.target_name, 
                                            dataset_name=self.dataset_name, 
                                            img_res=(self.img_rows, self.img_cols))
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))
        

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([img_A, img_B], validity)


    def generate_dataset(self):
        """
        pix2pix用の画像がtrain，val，testという3つのディレクトリすでに生成されているかどうか
        確かめたうえで各ディレクトリの画像の合計枚数を求めるメソッド．
        すでに，生成されている学習画像，検証用画像，テスト画像の再生成を行うことを防ぐために設けたメソッド

        各ディレクトリが存在したうえで
        画像の合計枚数が，self.image_num 枚 と一致していれば，画像がすでに生成されているものとして，画像を生成しない
        
        各ディレクトリが存在していない，または，画像の合計枚数が，self.image_num 枚 と一致しないとき
        DataGeneratorクラスのgenerate_dateメソッドによりpix2pix用の画像を生成する
        """
        partitions = ["train", "val", "test"]
        total_images = 0
        try:
          for partition in partitions:
              images_each_partition = os.listdir("../figure/foodimg128/{}/{}".format(self.dataset_name, partition))
              total_images += len(images_each_partition)
        except FileNotFoundError:
          self.data_generator.generate_data()

        if total_images == self.image_num:
          return
        else:
          self.data_generator.generate_data()
       

    def train(self, epochs, batch_size=1, sample_interval=50):
        start_time = datetime.datetime.now()
        
        # Generate data for pix2pix
        self.generate_dataset()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch+1, epochs,
                                                                                                      batch_i+1, self.data_loader.n_batches,
                                                                                                      d_loss[0], 100*d_loss[1],
                                                                                                      g_loss[0], elapsed_time))
                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                elif batch_i + 2  == self.data_loader.n_batches:
                    self.sample_images(epoch, batch_i)

    def sample_images(self, epoch, batch_i):
        os.makedirs('../result/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c, figsize=(10, 10))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("../result/{}/epoch{}_imageid{}.png".format(self.dataset_name, epoch+1, batch_i),
                    transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        if batch_i + 2 == self.data_loader.n_batches:
            fig.savefig("../result/{}/epoch{}_imageid{}.png".format(self.dataset_name, epoch+1, batch_i+2),
                        transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        plt.close()

In [ ]:
pix2pix = Pix2Pix()
pix2pix.train(epochs=200, batch_size=1, sample_interval=200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


[Epoch 1/200] [Batch 1/600] [D loss: 15.392680, acc:  39%] [G loss: 63.968002] time: 0:00:10.075164


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


ストリーミング出力は最後の 5000 行に切り捨てられました。
[Epoch 53/200] [Batch 194/600] [D loss: 0.001185, acc: 100%] [G loss: 33.180931] time: 1:47:55.419515
[Epoch 53/200] [Batch 195/600] [D loss: 0.002466, acc: 100%] [G loss: 40.515320] time: 1:47:55.605790
[Epoch 53/200] [Batch 196/600] [D loss: 0.001072, acc: 100%] [G loss: 36.307487] time: 1:47:55.784842
[Epoch 53/200] [Batch 197/600] [D loss: 0.001551, acc: 100%] [G loss: 19.359695] time: 1:47:55.970194
[Epoch 53/200] [Batch 198/600] [D loss: 0.004112, acc: 100%] [G loss: 29.265818] time: 1:47:56.149849
[Epoch 53/200] [Batch 199/600] [D loss: 0.002041, acc: 100%] [G loss: 25.559286] time: 1:47:56.333888
[Epoch 53/200] [Batch 200/600] [D loss: 0.001873, acc: 100%] [G loss: 25.814373] time: 1:47:56.517018
[Epoch 53/200] [Batch 201/600] [D loss: 0.002397, acc: 100%] [G loss: 31.365097] time: 1:47:56.707591
[Epoch 53/200] [Batch 202/600] [D loss: 0.003975, acc:  99%] [G loss: 24.107267] time: 1:47:59.836394
[Epoch 53/200] [Batch 203/600] [D loss: 0.002261, 